## <font size=4> *Modelación y Simulación*, 2024 </font>
## <font size=3 color='gray'> Samuel Chamalé </font>

# 1. (Modelo de Producción, Período único)

In [123]:
# Add the packages if not already installed
# import Pkg
# Pkg.add("JuMP")
# Pkg.add("HiGHS")

In [124]:
using JuMP, Clp

In [125]:
d1 = 800
d2 = 750
d3 = 600
d4 = 500

500

In [126]:
function solve_model(is_integer::Bool)
    # Define the model
    model = Model()

    # Decision variables
    if is_integer
        @variable(model, x1 >= 0, Int)  # Chamarras
        @variable(model, x2 >= 0, Int)  # Relleno de plumas
        @variable(model, x3 >= 0, Int)  # Pantalones
        @variable(model, x4 >= 0, Int)  # Guantes
    else
        @variable(model, x1 >= 0)  # Chamarras
        @variable(model, x2 >= 0)  # Relleno de plumas
        @variable(model, x3 >= 0)  # Pantalones
        @variable(model, x4 >= 0)  # Guantes
    end

    # Variables de holgura para la demanda no satisfecha
    @variable(model, s1 >= 0)  # Chamarras
    @variable(model, s2 >= 0)  # Relleno de plumas
    @variable(model, s3 >= 0)  # Pantalones
    @variable(model, s4 >= 0)  # Guantes

    # Variables de holgura para la capacidad no utilizada
    @variable(model, s5 >= 0)  # Corte
    @variable(model, s6 >= 0)  # Aislamiento
    @variable(model, s7 >= 0)  # Costura
    @variable(model, s8 >= 0)  # Empaque

    # Funcion objetivo
    @objective(model, Max,
        30x1 + 40x2 + 20x3 + 10x4 - 15s1 - 20s2 - 10s3 - 8s4
    )

    @constraint(model, x1 + s1 == d1)
    @constraint(model, x2 + s2 == d2)
    @constraint(model, x3 + s3 == d3)
    @constraint(model, x4 + s4 == d4)

    @constraint(model, 0.30x1 + 0.30x2 + 0.25x3 + 0.15x4 + s5 == 1000)  # Corte
    @constraint(model, 0.25x1 + 0.35x2 + 0.30x3 + 0.10x4 + s6 == 1000)  # Aislamiento
    @constraint(model, 0.45x1 + 0.50x2 + 0.40x3 + 0.22x4 + s7 == 1000)  # Costura
    @constraint(model, 0.15x1 + 0.15x2 + 0.10x3 + 0.05x4 + s8 == 1000)  # Empaque

    set_optimizer(model, HiGHS.Optimizer)

    optimize!(model)

    println(is_integer ? "\nCon restricciones de enteros:" : "\nSin restricciones de enteros:")
    println("Chamarras: ", value(x1))
    println("Relleno de plumas: ", value(x2))
    println("Pantalones: ", value(x3))
    println("Guantes: ", value(x4))
    println("\nFaltantes en la producción:")
    println("Chamarras faltantes: ", value(s1))
    println("Relleno de plumas faltantes: ", value(s2))
    println("Pantalones faltantes: ", value(s3))
    println("Guantes faltantes: ", value(s4))
    println("\nCapacidad no utilizada (horas):")
    println("Corte: ", value(s5))
    println("Aislamiento: ", value(s6))
    println("Costura: ", value(s7))
    println("Empaque: ", value(s8))
    println("\nUtilidad Total: \$", objective_value(model))
end

solve_model (generic function with 1 method)

In [127]:
solve_model(false)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-02, 1e+00]
  Cost   [8e+00, 4e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 1e+03]
Presolving model
1 rows, 5 cols, 5 nonzeros  0s
1 rows, 5 cols, 5 nonzeros  0s
Presolve : Reductions: rows 1(-7); columns 5(-7); elements 5(-23)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          1    -6.4625000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  6.4625000000e+04
HiGHS run time      :          0.00

 Sin restricciones de enteros:
Chamarras: 800.0
Relleno de plumas: 750.0
Pantalones: 387.5
Guantes: 500.0

Faltantes en la producción:
Chamarras faltantes: 0.0
Relleno de plumas faltantes: 0.0
Pantalones faltantes: 212.5
Guantes faltantes: 0.0



In [128]:
solve_model(true)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-02, 1e+00]
  Cost   [8e+00, 4e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 1e+03]
Presolving model
1 rows, 5 cols, 5 nonzeros  0s
1 rows, 5 cols, 5 nonzeros  0s
Objective function is integral with scale 0.333333

Solving MIP model with:
   1 rows
   5 cols (0 binary, 4 integer, 1 implied int., 0 continuous)
   5 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   71000           -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   64625           64595              0.05%        0      0      0         1     0.0s
 C       0       0         0   0.00%   64622           64610 